# Segmenting / Clustering Neighborhoods in Toronto Part 1

Import libraries/ packages

In [50]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# for webscraping import Beautiful Soup 
from bs4 import BeautifulSoup

import xml

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.10
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: //anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.10.3               |   py37hf985489_0         3.0 MB  conda-forge
    geographiclib-1.52         |     pyhd8ed1ab_0          35 KB  conda-forge
    geopy-2.2.0                |     pyhd8ed1ab_0          67 KB  conda-forge
    python_abi-3.7             |          2_cp37m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.52-

In [64]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'lxml')

In [76]:
table_post = soup.find('table')
fields = table_post.find_all('td')

postcode = []
borough = []
neighbourhood = []

for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i].text.strip())
    neighbourhood.append(fields[i].text.strip())
        
df_pc = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df_pc.columns = ['Postcode', 'Borough', 'Neighbourhood']

df_pc['Postcode'] = df_pc['Postcode'].str[:3]
df_pc['Borough'] = df_pc['Borough'].str[3:]
df_pc['Neighbourhood'] = df_pc['Neighbourhood'].str[3:]

df_pc.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M4A,North York(Victoria Village),North York(Victoria Village)
2,M7A,Queen's Park(Ontario Provincial Government),Queen's Park(Ontario Provincial Government)
3,M1B,Scarborough(Malvern / Rouge),Scarborough(Malvern / Rouge)
4,M4B,East York(Parkview Hill / Woodbine Gardens),East York(Parkview Hill / Woodbine Gardens)


In [77]:
df_pc['Borough'].replace('Not assigned', np.nan, inplace=True)
df_pc.dropna(subset=['Borough'], inplace=True)

df_pc['Neighbourhood'].replace('Not assigned', "Queen's Park", inplace=True)

df_pc

,Postcode,Borough,Neighbourhood
1,M4A,North York(Victoria Village),North York(Victoria Village)
2,M7A,Queen's Park(Ontario Provincial Government),Queen's Park(Ontario Provincial Government)
3,M1B,Scarborough(Malvern / Rouge),Scarborough(Malvern / Rouge)
4,M4B,East York(Parkview Hill / Woodbine Gardens),East York(Parkview Hill / Woodbine Gardens)
6,M1C,Scarborough(Rouge Hill / Port Union / Highland...,Scarborough(Rouge Hill / Port Union / Highland...
7,M4C,East York(Woodbine Heights),East York(Woodbine Heights)
9,M1E,Scarborough(Guildwood / Morningside / West Hill),Scarborough(Guildwood / Morningside / West Hill)
10,M4E,East Toronto(The Beaches),East Toronto(The Beaches)
12,M1G,Scarborough(Woburn),Scarborough(Woburn)
13,M4G,East York(Leaside),East York(Leaside)


In [79]:
df_pcn = df_pc.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_pcn.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_pcn

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough(Malvern / Rouge),Scarborough(Malvern / Rouge)
1,M1C,Scarborough(Rouge Hill / Port Union / Highland...,Scarborough(Rouge Hill / Port Union / Highland...
2,M1E,Scarborough(Guildwood / Morningside / West Hill),Scarborough(Guildwood / Morningside / West Hill)
3,M1G,Scarborough(Woburn),Scarborough(Woburn)
4,M1H,Scarborough(Cedarbrae),Scarborough(Cedarbrae)
5,M1J,Scarborough(Scarborough Village),Scarborough(Scarborough Village)
6,M1K,Scarborough(Kennedy Park / Ionview / East Birc...,Scarborough(Kennedy Park / Ionview / East Birc...
7,M1L,Scarborough(Golden Mile / Clairlea / Oakridge),Scarborough(Golden Mile / Clairlea / Oakridge)
8,M1M,Scarborough(Cliffside / Cliffcrest / Scarborou...,Scarborough(Cliffside / Cliffcrest / Scarborou...
9,M1N,Scarborough(Birch Cliff / Cliffside West),Scarborough(Birch Cliff / Cliffside West)


In [82]:
df_geo = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')
df_geo.columns = ['Postcode', 'Latitude', 'Longitude']

In [83]:
df_pos = pd.merge(df_pcn, df_geo, on=['Postcode'], how='inner')

df_toro = df_pos[['Borough', 'Neighbourhood', 'Postcode', 'Latitude', 'Longitude']].copy()

df_toro.head()

,Borough,Neighbourhood,Postcode,Latitude,Longitude
0,Scarborough(Malvern / Rouge),Scarborough(Malvern / Rouge),M1B,43.806686,-79.194353
1,Scarborough(Rouge Hill / Port Union / Highland...,Scarborough(Rouge Hill / Port Union / Highland...,M1C,43.784535,-79.160497
2,Scarborough(Guildwood / Morningside / West Hill),Scarborough(Guildwood / Morningside / West Hill),M1E,43.763573,-79.188711
3,Scarborough(Woburn),Scarborough(Woburn),M1G,43.770992,-79.216917
4,Scarborough(Cedarbrae),Scarborough(Cedarbrae),M1H,43.773136,-79.239476


In [85]:
address = 'Toronto, Canada'

geolocator =  Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of the City of Toronto are 43.6534817, -79.3839347.


In [89]:
# create map of New York using latitude and longitude values
map_of_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toro['Latitude'], df_toro['Longitude'], df_toro['Borough'], df_toro['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_of_toronto)  
    
map_of_toronto